# Data Streaming Invasive Alien Species Digital Twin Outputs

**Author**: [Taimur Khan](https://www.ufz.de/index.php?en=49404), Helmholtz Centre for Environmental Research - UFZ, Germany

**Date**: 2025-02-20

## Purpose

The purpose of this notebook is to demonstrate how to stream the outputs of the Invasive Alien Species Digital Twin from the [BioDT OPenDAP service](http://opendap.biodt.eu). The outputs are streamed in real-time using `rioxarray` and visualized using the `matplotlib` library.

The notebook has been parameterized using Jupyter Widgets to allow the user to select the desired habitat, climate model, climate scenario, and time period parameters for querying the desired dataset. 

The notebook is divided into the following sections:
- [1. Import Libraries](#1.-Import-Libraries)
- [2. Define Parameters](#2.-Define-Parameters)
- [3. Query URL](#3.-Query-Data)
- [4. Stream Data](#4.-Stream-Data)
- [5. Visualize Data](#5.-Visualize-Data)
- [6. Download Data (optional)](#6.-Download)

## 1. Import Libraries

If you do not have the required libraries installed, you can install them by running the following cell:

In [5]:
!pip install rioxarray matplotlib pandas matplotlib_scalebar ipywidgets

  Using cached rioxarray-0.15.0-py3-none-any.whl (53 kB)
  Using cached matplotlib-3.9.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.3 MB)
  Using cached pandas-2.2.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Using cached matplotlib_scalebar-0.9.0-py3-none-any.whl (16 kB)
  Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)
  Using cached pyproj-3.6.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.3 MB)
  Using cached xarray-2024.7.0-py3-none-any.whl (1.2 MB)
  Using cached rasterio-1.4.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (22.2 MB)
  Using cached numpy-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
  Using cached fonttools-4.56.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.6 MB)
  Using cached contourpy-1.3.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (321 kB)
  Using cached pillow-11.1.0-cp39-cp39-manylinux_2_28_x86_64.whl (4.5 MB)
  Using cached 

Otherwise just import the following:

In [6]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import rioxarray
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.patches as mpatches

In [7]:
# A-Selecte Habitat
url = "http://opendap.biodt.eu/ias-pdt/0/outputs/key.csv"
df_hab = pd.read_csv(url)

# Display the dropdown widget
print("Select a habitat type from the dropdown list:")


# Get the corresponding value for hab_abb for the selected hab_name
selected_hab_abb = '1'
param_habitat_type = 'human_maintained_grasslands'
conf_data_path = '/tmp/data/'
selected_hab_abb = str(df_hab[df_hab["hab_name"] == param_habitat_type]["hab_abb"].values[0])

print(f"Selected Habitat Abbreviation: {selected_hab_abb}")



Select a habitat type from the dropdown list:
Selected Habitat Abbreviation: 4b


### Select Climate Model, Scenario, and Species

In [8]:
# B-Load the remote txt file as dataframe
url_txt = f"http://opendap.biodt.eu/ias-pdt/0/outputs/hab{selected_hab_abb}/predictions/Prediction_Summary_Shiny.txt"
df_mod = pd.read_csv(url_txt, sep="\t")

# Create dropdown widgets with added space
habitat_dropdown = widgets.Dropdown(
    options=df_mod[
        "hab_name"
    ].unique(),  # Replace 'column_name' with the actual column name you want to use
    description="Habitat Type:",
    disabled=False,
    layout=widgets.Layout(margin="0 0 0 0px"),
)

climate_model_dropdown = widgets.Dropdown(
    options=df_mod["climate_model"].unique(),
    description="Climate Model:",
    disabled=False,
    layout=widgets.Layout(margin='0 0 0 0px')
)

climate_model_dropdown.options = list(climate_model_dropdown.options) + ["Ensemble"]


climate_scenario_dropdown = widgets.Dropdown(
    options=df_mod["climate_scenario"].unique(),
    description="Climate Scenario:",
    disabled=False,
    layout=widgets.Layout(margin='0 0 0 0px')
)

time_period_dropdown = widgets.Dropdown(
    options=df_mod["time_period"].unique(),
    description="Time Period:",
    disabled=False,
    layout=widgets.Layout(margin='0 0 0 0px')
)

species_name_dropdown = widgets.Dropdown(
    options=df_mod["species_name"].dropna().unique(),
    description="Species Name:",
    disabled=False,
) 

param_habitat_name =  'Human maintained grasslands'
param_climate_model = 'Current'
param_climate_scenario = 'Current'
param_time_period = '1981-2010'
param_species_name = 'Agave americana' 


conf_x =  0.95
conf_y =  0.95
conf_arrow_length = 0.1


Human maintained grasslands


## 3. Query URL

In [5]:
# C-Query URL
url_txt = f"http://opendap.biodt.eu/ias-pdt/0/outputs/hab{selected_hab_abb}/predictions/Prediction_Summary_Shiny.txt"
df_mod = pd.read_csv(url_txt, sep="\t")

hab_num = df_mod[df_mod["hab_name"] == param_habitat_name]["hab_abb"].values[0]
tif_path_mean = df_mod[
    (df_mod["hab_abb"] == hab_num) &
    (df_mod["climate_model"] == param_climate_model) &
    (df_mod["climate_scenario"] == param_climate_scenario) &
    (df_mod["time_period"] == param_time_period) &
    (df_mod["species_name"] == param_species_name)
]["tif_path_mean"].values[0]

tif_path_sd = df_mod[
    (df_mod["hab_abb"] == hab_num)
    & (df_mod["climate_model"] == param_climate_model)
    & (df_mod["climate_scenario"] == param_climate_scenario)
    & (df_mod["time_period"] == param_time_period)
    & (df_mod["species_name"] == param_species_name)
]["tif_path_sd"].values[0]

tif_path_cov = df_mod[
    (df_mod["hab_abb"] == hab_num)
    & (df_mod["climate_model"] == param_climate_model)
    & (df_mod["climate_scenario"] == param_climate_scenario)
    & (df_mod["time_period"] == param_time_period)
    & (df_mod["species_name"] == param_species_name)
]["tif_path_cov"].values[0]

tif_path_anomaly = df_mod[
    (df_mod["hab_abb"] == hab_num)
    & (df_mod["climate_model"] == param_climate_model)
    & (df_mod["climate_scenario"] == param_climate_scenario)
    & (df_mod["time_period"] == param_time_period)
    & (df_mod["species_name"] == param_species_name)
]["tif_path_anomaly"].values[0]

mean_url = f"http://opendap.biodt.eu/ias-pdt/0/outputs/hab{hab_num}/predictions/{tif_path_mean}"
sd_url = f"http://opendap.biodt.eu/ias-pdt/0/outputs/hab{hab_num}/predictions/{tif_path_sd}"
cov_url = f"http://opendap.biodt.eu/ias-pdt/0/outputs/hab{hab_num}/predictions/{tif_path_cov}"
anomaly_url = f"http://opendap.biodt.eu/ias-pdt/0/outputs/hab{hab_num}/predictions/{tif_path_anomaly}"

In [6]:
print(mean_url)
print(sd_url)
print(cov_url)
print(anomaly_url)


http://opendap.biodt.eu/ias-pdt/0/outputs/hab4b/predictions/Current/Sp_0025_mean.tif
http://opendap.biodt.eu/ias-pdt/0/outputs/hab4b/predictions/Current/Sp_0025_sd.tif
http://opendap.biodt.eu/ias-pdt/0/outputs/hab4b/predictions/Current/Sp_0025_cov.tif
http://opendap.biodt.eu/ias-pdt/0/outputs/hab4b/predictions/nan


## 4. Stream Data

## 5. Visualize Data

In [7]:
# D-Plot Mean
fig, ax = plt.subplots(figsize=(10, 8))
data_mean = rioxarray.open_rasterio(mean_url)
data_mean.plot(ax=ax, cmap="Spectral")
image_name = f"Mean Species Distribution for {param_species_name} in {param_habitat_name} for {param_time_period} and {param_climate_model} {param_climate_scenario}"
plt.title(image_name)

# Add grid lines
ax.grid(True, linestyle='--', linewidth=0.5)

# Add a scale bar
scalebar = ScaleBar(1, location='upper right')  # 1 pixel = 1 unit
ax.add_artist(scalebar)

# Add a north arrow
ax.annotate('N', xy=(conf_x, conf_y), xytext=(conf_x, conf_y-conf_arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=12,
            xycoords=ax.transAxes)

# plt.show()

# Save the figure
plt.savefig(conf_data_path+image_name)
plt.close()

Warning 1: HTTP response code on http://opendap.biodt.eu/ias-pdt/0/outputs/hab4b/predictions/Current/Sp_0025_mean.tif.msk: 502
Warning 1: HTTP response code on http://opendap.biodt.eu/ias-pdt/0/outputs/hab4b/predictions/Current/Sp_0025_mean.tif.MSK: 502
/opt/conda/lib/python3.9/site-packages/matplotlib_scalebar/scalebar.py:457: UserWarning: Drawing scalebar on axes with unequal aspect ratio; either call ax.set_aspect(1) or suppress the warning with rotation='horizontal-only'.
  warnings.warn(


In [8]:
# E-Plot stdev
fig, ax = plt.subplots(figsize=(10, 8))
data_sd = rioxarray.open_rasterio(sd_url)
data_sd.plot(ax=ax, cmap="Spectral")
image_name = f"Standard Deviation of {param_species_name} distribution for {param_climate_model} {param_climate_scenario} {param_time_period}"
plt.title(image_name)

# Add grid lines
ax.grid(True, linestyle='--', linewidth=0.5)

# Add a scale bar
scalebar = ScaleBar(1, location='upper right')  # 1 pixel = 1 unit
ax.add_artist(scalebar)

# Add a north arrow
ax.annotate('N', xy=(conf_x, conf_y), xytext=(conf_x, conf_y-conf_arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=12,
            xycoords=ax.transAxes)
plt.savefig(conf_data_path+image_name)
plt.close()



Warning 1: HTTP response code on http://opendap.biodt.eu/ias-pdt/0/outputs/hab4b/predictions/Current/Sp_0025_sd.tif.msk: 502
Warning 1: HTTP response code on http://opendap.biodt.eu/ias-pdt/0/outputs/hab4b/predictions/Current/Sp_0025_sd.tif.MSK: 502
/opt/conda/lib/python3.9/site-packages/matplotlib_scalebar/scalebar.py:457: UserWarning: Drawing scalebar on axes with unequal aspect ratio; either call ax.set_aspect(1) or suppress the warning with rotation='horizontal-only'.
  warnings.warn(


In [9]:
# F-Plot Coefficient
fig, ax = plt.subplots(figsize=(10, 8))
data_cov = rioxarray.open_rasterio(cov_url)
data_cov.plot(ax=ax, cmap="Spectral")
image_name = f"Coefficient of Variation of {param_species_name} distribution for {param_climate_model} {param_climate_scenario} {param_time_period}"
plt.title(image_name)

# Add grid lines
ax.grid(True, linestyle='--', linewidth=0.5)

# Add a scale bar
scalebar = ScaleBar(1, location='upper right')  # 1 pixel = 1 unit
ax.add_artist(scalebar)

# Add a north arrow
ax.annotate('N', xy=(conf_x, conf_y), xytext=(conf_x, conf_y-conf_arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=12,
            xycoords=ax.transAxes)

plt.savefig(conf_data_path+image_name)
plt.close()

Warning 1: HTTP response code on http://opendap.biodt.eu/ias-pdt/0/outputs/hab4b/predictions/Current/Sp_0025_cov.tif.msk: 502
Warning 1: HTTP response code on http://opendap.biodt.eu/ias-pdt/0/outputs/hab4b/predictions/Current/Sp_0025_cov.tif.MSK: 502
/opt/conda/lib/python3.9/site-packages/matplotlib_scalebar/scalebar.py:457: UserWarning: Drawing scalebar on axes with unequal aspect ratio; either call ax.set_aspect(1) or suppress the warning with rotation='horizontal-only'.
  warnings.warn(


In [10]:
# G-Ploat Anomaly
if "nan" not in anomaly_url:
    data_anomaly = rioxarray.open_rasterio(anomaly_url)
    fig, ax = plt.subplots(figsize=(10, 8))
    data_anomaly.plot(ax=ax, cmap="Spectral")
    image_name = f"Anomaly of {param_species_name} distribution for {param_climate_model} {param_climate_scenario} {param_time_period}"
    plt.title(image_name)

    # Add grid lines
    ax.grid(True, linestyle='--', linewidth=0.5)

    # Add a scale bar
    scalebar = ScaleBar(1, location='upper right')  # 1 pixel = 1 unit
    ax.add_artist(scalebar)

    # Add a north arrow
    ax.annotate('N', xy=(conf_x, conf_y), xytext=(conf_x, conf_y-conf_arrow_length),
                arrowprops=dict(facecolor='black', width=5, headwidth=15),
                ha='center', va='center', fontsize=12,
                xycoords=ax.transAxes)

    # plt.show()
    plt.savefig(conf_data_path+image_name)
    plt.close()
else:
    print("Anomaly map does not exist for the current selection")

Anomaly map does not exist for the current selection
